<a href="https://colab.research.google.com/github/titusjscott/multi-layer-perceptron/blob/main/activity_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.utils import to_categorical
from google.colab import files

In [2]:
uploaded = files.upload()

train = pd.read_csv('activity_recognition_train.csv')

Saving activity_recognition_train.csv to activity_recognition_train.csv


In [3]:
train.shape

(7352, 563)

In [4]:
uploaded = files.upload()

test = pd.read_csv('activity_recognition_test.csv')

Saving activity_recognition_test.csv to activity_recognition_test.csv


In [5]:
test.shape

(2947, 563)

In [6]:
# combine train and test dataframes into one to ensure consistent cleaning of data

df = pd.concat([train, test])

df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [7]:
df.shape

(10299, 563)

In [8]:
# Change the Activity column to numerical data

le = LabelEncoder()
df["Activity"] = le.fit_transform(df["Activity"])

In [9]:
# Check to find out if there are other columns that need to be converted to numeric

non_numeric_cols = df.select_dtypes(exclude=['number']).columns.tolist()

if len(non_numeric_cols) > 0:
    print(f"The following columns are not numeric: {non_numeric_cols}")
else:
    print("All columns are numeric")

All columns are numeric


In [11]:
# Separate input features (X) and target variable (y)
X = df.drop('Activity', axis=1)
y = df['Activity']

# Split the data into training and testing sets (use an appropriate test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Build an Autoencoder

In [24]:
# Define the input shape of the autoencoder
input_shape = (562,)

# Define the number of classes
num_classes = 6

# Define the size of the encoded representation
encoding_dim = 64

# Define the input layer
input_layer = Input(shape=input_shape)

# Convert the labels to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)
y_train_categorical = to_categorical(y_train)

# Define the encoder layers
encoder_layer_1 = Dense(256, activation='relu')(input_layer)
encoder_layer_2 = Dense(128, activation='relu')(encoder_layer_1)
encoder_layer_3 = Dense(encoding_dim, activation='relu')(encoder_layer_2)

# Define the decoder layers
decoder_layer_1 = Dense(128, activation='relu')(encoder_layer_3)
decoder_layer_2 = Dense(256, activation='relu')(decoder_layer_1)
decoder_layer_3 = Dense(input_shape[0], activation='sigmoid')(decoder_layer_2)

# Define the autoencoder model
autoencoder = Model(input_layer, decoder_layer_3)

# Define the encoder model
encoder_model = Model(input_layer, encoder_layer_3)

# Define the classifier model
classifier_model = Sequential()
classifier_model.add(Dense(64, activation='relu', input_dim=64))
classifier_model.add(Dense(32, activation='relu'))
classifier_model.add(Dense(6, activation='softmax'))


# Compile the models
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
classifier_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Fit the autoencoder model
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(X_test, X_test))

# Extract the encoded representation of the data
encoded_data = encoder_model.predict(X_train)

encoded_train = encoder_model.predict(X_train)
encoded_test = encoder_model.predict(X_test)

# Fit the classifier model using the encoded data
classifier_model.fit(encoded_train, y_train_categorical,
                     epochs=50,
                     batch_size=64,
                     validation_split=0.2)


Epoch 1/50
65/65 [==============================] - 2s 20ms/step - loss: -27438.7637 - val_loss: -281665.8750
Epoch 2/50
65/65 [==============================] - 1s 17ms/step - loss: -19370666.0000 - val_loss: -100269240.0000
Epoch 3/50
65/65 [==============================] - 1s 13ms/step - loss: -894834112.0000 - val_loss: -2933031936.0000
Epoch 4/50
65/65 [==============================] - 1s 13ms/step - loss: -11661725696.0000 - val_loss: -28717027328.0000
Epoch 5/50
65/65 [==============================] - 1s 13ms/step - loss: -76197437440.0000 - val_loss: -155569127424.0000
Epoch 6/50
65/65 [==============================] - 1s 14ms/step - loss: -326244663296.0000 - val_loss: -587008114688.0000
Epoch 7/50
65/65 [==============================] - 1s 13ms/step - loss: -1061393989632.0000 - val_loss: -1738962239488.0000
Epoch 8/50
65/65 [==============================] - 1s 13ms/step - loss: -2847522947072.0000 - val_loss: -4342035251200.0000
Epoch 9/50
65/65 [======================

In [25]:
from sklearn.metrics import classification_report


# Generate predictions for the test set
y_pred_prob = classifier_model.predict(encoded_test)

# Convert the probabilities to class labels
y_pred_labels = y_pred_prob.argmax(axis=1)

# Generate a classification report
print(classification_report(y_test, y_pred_labels))





65/65 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       406
           1       0.00      0.00      0.00       377
           2       0.17      1.00      0.29       354
           3       0.00      0.00      0.00       369
           4       1.00      0.00      0.01       284
           5       0.00      0.00      0.00       270

    accuracy                           0.17      2060
   macro avg       0.20      0.17      0.05      2060
weighted avg       0.17      0.17      0.05      2060



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout


# Define a function to create the MLP model
def create_model(hidden_layer_sizes=[64, 32], dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], activation='relu', input_shape=(encoded_train.shape[1],)))
    for size in hidden_layer_sizes[1:]:
        model.add(Dense(size, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(6, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create a KerasClassifier object from the create_model function
clf = KerasClassifier(build_fn=create_model, verbose=0)

# Define a grid of hyperparameters to search
param_grid = {
    'hidden_layer_sizes': [[64, 32], [128, 64, 32]],
    'dropout_rate': [0.2, 0.5],
    'optimizer': ['adam', 'rmsprop']
}

# Create a GridSearchCV object to search for the best hyperparameters
grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_result = grid.fit(encoded_train, y_train_categorical)

# Print the best hyperparameters and the corresponding accuracy
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


<ipython-input-27-0aead0f16c18>:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.186672 using {'dropout_rate': 0.2, 'hidden_layer_sizes': [64, 32], 'optimizer': 'rmsprop'}


In [28]:
classifier_model = Sequential()
classifier_model.add(Dense(64, activation='relu', input_dim=64))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(32, activation='relu'))
classifier_model.add(Dense(6, activation='softmax'))

classifier_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

classifier_model.fit(encoded_train, y_train_categorical,
                     epochs=100,
                     batch_size=64,
                     validation_split=0.2)



# Generate predictions for the test set
y_pred_prob = classifier_model.predict(encoded_test)

# Convert the probabilities to class labels
y_pred_labels = y_pred_prob.argmax(axis=1)

# Generate a classification report
print(classification_report(y_test, y_pred_labels))

Epoch 1/100
103/103 [==============================] - 1s 4ms/step - loss: 85989528.0000 - accuracy: 0.1575 - val_loss: 1.7951 - val_accuracy: 0.1323
Epoch 2/100
103/103 [==============================] - 0s 2ms/step - loss: 1.7926 - accuracy: 0.1560 - val_loss: 1.7907 - val_accuracy: 0.1899
Epoch 3/100
103/103 [==============================] - 0s 3ms/step - loss: 1.7888 - accuracy: 0.1836 - val_loss: 1.7880 - val_accuracy: 0.1899
Epoch 4/100
103/103 [==============================] - 0s 3ms/step - loss: 1.7881 - accuracy: 0.1815 - val_loss: 1.7867 - val_accuracy: 0.1857
Epoch 5/100
103/103 [==============================] - 0s 2ms/step - loss: 1.7868 - accuracy: 0.1877 - val_loss: 1.7859 - val_accuracy: 0.1857
Epoch 6/100
103/103 [==============================] - 0s 3ms/step - loss: 1.7887 - accuracy: 0.1883 - val_loss: 1.7855 - val_accuracy: 0.1857
Epoch 7/100
103/103 [==============================] - 0s 2ms/step - loss: 1.7866 - accuracy: 0.1892 - val_loss: 1.7852 - val_accuracy:

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from scipy.stats import randint as sp_randint

def create_model(optimizer=Adam(), neurons=32):
    classifier_model = Sequential()
    classifier_model.add(Dense(neurons, activation='relu', input_dim=encoding_dim))
    classifier_model.add(Dense(num_classes, activation='softmax'))
    classifier_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier_model

# Create a KerasClassifier from the model
clf = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters to search
params = {
    'optimizer': ['adam', 'rmsprop'],
    'neurons': sp_randint(32, 256),
}

# Define the randomized search
random_search = RandomizedSearchCV(clf, param_distributions=params, n_iter=10, cv=3)

# Fit the randomized search
random_search.fit(encoded_train, y_train_categorical, epochs=50, batch_size=64, validation_split=0.2)

# Print the best hyperparameters
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))

<ipython-input-38-de1795cfe2f4>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.184972 using {'neurons': 42, 'optimizer': 'rmsprop'}
